In [303]:
import pandas as pd

## Load the unlabelled dataset ##

In [304]:
df = pd.read_csv('data/from-vpn/AllData.csv')
df.index

c:\users\omri\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


RangeIndex(start=0, stop=19027, step=1)

In [305]:
features_cols = ['GridRows', 'GridColumns', 'NumOfAgents', 'NumOfObstacles', 'BranchingFactor', 'ObstacleDensity',
            'AvgDistanceToGoal', 'MaxDistanceToGoal', 'MinDistanceToGoal', 'AvgStartDistances', 'AvgGoalDistances',
            'PointsAtSPRatio']
runtime_cols = list(df.filter(like="Runtime"))
relev_cols = runtime_cols + features_cols + ['Y']

In [306]:
runtime_cols

['A*+OD+ID Runtime',
 'MA-CBS-Global-10/(EPEA*/SIC) choosing the first conflict in CBS nodes Runtime',
 'Basic-CBS/(A*/SIC)+ID Runtime',
 'ICTS 3E +ID Runtime',
 'EPEA*+ID Runtime',
 'CBS/(A*/SIC) + BP + PC without smart tie breaking using Dynamic Lazy Open List with Heuristic MVC of Cardinal Conflict Graph Heuristic Runtime']

## Create Y Column with argmin on runtime_cols ##

In [307]:
df[runtime_cols]

,A*+OD+ID Runtime,MA-CBS-Global-10/(EPEA*/SIC) choosing the first conflict in CBS nodes Runtime,Basic-CBS/(A*/SIC)+ID Runtime,ICTS 3E +ID Runtime,EPEA*+ID Runtime,CBS/(A*/SIC) + BP + PC without smart tie breaking using Dynamic Lazy Open List with Heuristic MVC of Cardinal Conflict Graph Heuristic Runtime
0,75.7295,48.8441,2.4742,2.6185,1.3487,54.4875
1,5.2334,0.1354,1.9447,1.8911,1.8622,0.1684
2,6.0557,0.2008,6.1211,5.6710,5.5621,0.2337
3,10.5929,0.3776,10.5565,10.3796,10.5652,1.5976
4,13.5519,0.4698,13.5414,13.6807,13.7112,0.5191
5,26.4102,0.7052,27.8298,27.6709,28.6532,0.7430
6,42.9793,1.2086,44.5217,36.8707,38.4553,0.9812
7,49.5234,1.1268,50.2001,46.4985,46.2204,1.1771
8,66.7485,1.3442,70.1400,69.1274,69.0126,1.4591
9,79.8633,1.5595,79.8691,80.6823,80.1122,1.7848


In [308]:
Y = df[runtime_cols].idxmin(axis=1)
df['Y'] = Y

In [309]:
df.iloc[400][runtime_cols], df.iloc[400]['Y'] # SANITY CHECK

(A*+OD+ID Runtime                                                                                                                                  26712.6
 MA-CBS-Global-10/(EPEA*/SIC) choosing the first conflict in CBS nodes Runtime                                                                      300000
 Basic-CBS/(A*/SIC)+ID Runtime                                                                                                                     27318.6
 ICTS 3E +ID Runtime                                                                                                                               30960.3
 EPEA*+ID Runtime                                                                                                                                  26978.1
 CBS/(A*/SIC) + BP + PC without smart tie breaking using Dynamic Lazy Open List with Heuristic MVC of Cardinal Conflict Graph Heuristic Runtime    3496.65
 Name: 400, dtype: object,
 'CBS/(A*/SIC) + BP + PC without smart tie 

In [310]:
def runtime_to_success(col):
    splitted = col.split()
    splitted[-1] = "Success"
    return " ".join(splitted)

runtime_to_success("Y Runtime")

'Y Success'

In [311]:
df['Y Success'] = df.apply(lambda x:x[runtime_to_success(x['Y'])], axis=1)

In [312]:
df['Y Runtime'] = df.apply(lambda x:x[x['Y']], axis=1)

In [313]:
df['Y'].value_counts()

CBS/(A*/SIC) + BP + PC without smart tie breaking using Dynamic Lazy Open List with Heuristic MVC of Cardinal Conflict Graph Heuristic Runtime    9312
MA-CBS-Global-10/(EPEA*/SIC) choosing the first conflict in CBS nodes Runtime                                                                     4853
ICTS 3E +ID Runtime                                                                                                                               2216
EPEA*+ID Runtime                                                                                                                                  1953
A*+OD+ID Runtime                                                                                                                                   518
Basic-CBS/(A*/SIC)+ID Runtime                                                                                                                      175
Name: Y, dtype: int64

## Add Sparsity feature 

In [314]:
df['GridSize'] = df['GridRows'] * df['GridColumns']

In [315]:
df['Sparsity'] = df.apply(lambda x: x['NumOfAgents']/(x['GridSize']-x['NumOfObstacles']),axis=1)

In [316]:
features_cols.append('GridSize')
features_cols.append('Sparsity')


In [317]:
features_cols.append('Y')

## Save labelled data to file

In [318]:
df.to_csv("data/from-vpn/AllData-labelled.csv", index=False)

## Adding ranking (in order to train LTR models) ##

# In order to add 'bridges' feature use CreateGridFromMapFile notebook